<a href="https://colab.research.google.com/github/DmtrRm/IAAC2024_tutorials/blob/main/kaggle_complete2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIBRARIES IMPORT**

In [254]:
import calendar
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

**FUNCTIONS THAT WILL BE USED**

In [255]:
def encode_binary_categorical(data_set):
  binary_Lbls=[]
  categor_Lbls=[]

  for c in data_set.columns:

    if len(data_set[c].unique())<=3 and any(x=="Yes" for x in data_set[c].unique()):
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=1-res
      binary_Lbls.append(c)

    elif data_set[c].dtype=="object" :
      le=LabelEncoder()
      res =le.fit_transform(data_set[c])
      data_set[c]=res
      categor_Lbls.append(c)

  return categor_Lbls

In [256]:
def find_pair_list(corralation,cor_value):


  pair_list=[]
  x=corralation.shape[0]
  y=corralation.shape[1]

  for i in range(0,x):
    for j in range(0,y):
      if (corralation.iloc[i,j]>=cor_value or corralation.iloc[i,j]<=(-cor_value)) and corralation.iloc[i,j]!=1 :
        pair_list=pair_list+[(sorted([corralation.columns[i], corralation.columns[j]]))]

  groups = []

  for pair in pair_list:
    added_to_group = False
    for group in groups:
      if any(variable in group for variable in pair):
        group.extend(pair)
        group = list(set(group))
        added_to_group = True
        break
    if not added_to_group:
      groups.append(pair)

  result = [list(set(group)) for group in groups]
  return result

In [257]:
def data_cleaning(i):

## dropping same value clm
  for c in i.columns:
   if len(i[c].unique())==1:
    i=i.drop([c], axis=1)

## encoding
  cLbls_i=encode_binary_categorical(i)

## droping dublicates
  result_i=find_pair_list(i.corr(),0.9)
  print(result_i)
  columns_to_drop = sum([x[0:-1] for x in result_i],[])
  i=i.drop(columns=columns_to_drop)

## filling the NaN
  cols_to_drop = []
  for c in i.columns:
    if len(i[c].isna().unique())>1:
      cols_to_drop.append(c)
  print(cols_to_drop)

  if not cols_to_drop==False:
    for n in cols_to_drop:
      X = i.loc[i[n].isna()==False].drop(columns=cols_to_drop)
      Y = i.loc[i[n].isna()==False][n]
      missing_X = i.loc[i[n].isna()].drop(columns=cols_to_drop)
      tree = DecisionTreeRegressor()
      tree.fit(X, Y)

      zipped = [(x, i) for i,x in sorted(zip(tree.feature_importances_, X), reverse=True)]

      i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)

  return i

**DATA UPLOADING**

In [258]:
metadata = pd.read_csv("/content/DATA/metadata.csv", index_col=0)
meters = pd.read_csv("/content/DATA/meters.csv", index_col=0)
test = pd.read_csv("/content/DATA/test.csv", index_col=0)
train = pd.read_csv("/content/DATA/train.csv", index_col=0)
weather = pd.read_csv("/content/DATA/weather.csv", index_col=0)

In [259]:
weather[['day', 'time']] = weather['timestamp'].str.split(' ', expand=True)

**DATA CLEANING**

In [241]:
weather["day"] = weather["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
weather["year"] = weather["day"].apply(lambda s: s.year)
weather["month"] = weather["day"].apply(lambda s: s.month)
weather["day_number"] = weather["day"].apply(lambda s: s.day)
weather["weekday"] = weather["day"].apply(lambda s: s.weekday() in [5,6])

In [242]:
train["day"] = train["day"].apply(lambda s: datetime.strptime(s, "%Y-%m-%d"))
train["year"] = train["day"].apply(lambda s: s.year)
train["month"] = train["day"].apply(lambda s: s.month)
train["day_number"] = train["day"].apply(lambda s: s.day)
train["weekday"] = train["day"].apply(lambda s: s.weekday() in [5,6])

In [260]:
metadata = data_cleaning(metadata)
train = data_cleaning(train)
weather = data_cleaning(weather)

[['sqft', 'sqm'], ['heatingtype', 'lng', 'timezone', 'lat']]
['site_id_kaggle', 'yearbuilt', 'numberoffloors', 'occupants']
[]
[]


<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-i

[['timestamp', 'day']]
['airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']


<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-input-257-6aa915e1368b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i[n].loc[i[n].isna()] = tree.predict(missing_X).astype(int)
<ipython-i

**DATA MERGING**

In [262]:
full_df = train.merge(metadata, on='building_id', how='left')

In [263]:
full_df=full_df.merge(weather, how="left", on=["day", "site_id"])